In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import glob
import time 
import numpy as np
from itertools import combinations
from mwc_class import getMVWs
from mwc_functions import *
from optimization_functions import *
from power_indice_functions import * 


In [ ]:
start_time=time.time()
krohn_data = getMVWs('krohn.csv', name='krohn_example', save_results=False,verify_mwcs=True,encoding='UTF-8',delimiter=',')
prelims = krohn_data.preliminaries()
country_MIW = krohn_data.minimal_voting_weights_pipeline()
end_time=time.time()
duration_Verify=end_time-start_time

print(duration_Verify)

# Create Functions that can calculate Banzhaf Index 

first get a list of mwc´s and parties for a year:

In [ ]:
min_win_coals_year=minimal_winning_coalitions_for_a_year(krohn_data.minimal_winning_coalitions,'2001')
relevant_parties_for_a_year=relevant_parties_for_a_year(krohn_data.minimal_winning_coalitions,'2001')

create a function that can create a list of any 2 mwc unions 

In [ ]:
mwc_sets = [set(mwc.split('+')) for mwc in min_win_coals_year]

In [ ]:

def union_of_two_mwcs_as_sets(min_win_coals_year):
    mwc_sets = [set(mwc.split('+')) for mwc in min_win_coals_year]
    combined_coalitions_sets = []
    for mwc1, mwc2 in combinations(mwc_sets, 2):
        combined_set = mwc1.union(mwc2) 
        combined_coalitions_sets.append(combined_set)
    return combined_coalitions_sets


In [ ]:
test_set_2 = union_of_two_mwcs_as_sets(min_win_coals_year)

extend this to n-mwc-unions:

In [ ]:
def union_of_n_mwcs_as_sets(min_win_coals_year, n):
    mwc_sets = [set(mwc.split('+')) for mwc in min_win_coals_year]
    combined_coalitions_sets = []
    for mwc_combination in combinations(mwc_sets, n):
        combined_set = set().union(*mwc_combination)
        combined_coalitions_sets.append(combined_set)
    return combined_coalitions_sets

In [ ]:
test_set_4=union_of_n_mwcs_as_sets(min_win_coals_year,4)

now the opus magnum of this code: 

a function which creates all n-unions for n= number of mwcs \\

for any party consider only sets in which that party is a member \\

for those sets calculate $2^{n-\#\{\bigcup(\dots)\}}$ \\

sum over all those and add or substract them from BS_i for all n \\

this generally follows the steps of the example in "Power indices and minimal winning coalitions",Werner Kirsch, Jessica Langner,"Social Choice and Welfare, Vol. 34, No. 1 (January 2010), pp. 33-46 (14 pages)"
https://www.jstor.org/stable/41108037

In [ ]:
def BS_w(min_win_coals_year,relevant_parties): # to stay in notation of the paper, indicates the Banzhaf score of a voter w -> in our case score of a party 
    steps=len(min_win_coals_year)
    BS={}
    coalition_sets = {}
    for i in range(1,steps+1): 
       coalition_sets[i]= union_of_n_mwcs_as_sets(min_win_coals_year, i)
    for party_i in relevant_parties:
        BS_i = 0
        for i in range(1,steps+1): #We need a step for any value in steps (1,2,...,len(min_win_coals_year))
            coalition_sets_with_i = [coalition_set for coalition_set in coalition_sets[i] if party_i in coalition_set] #in words: list every coalition set from coalition_sets[i] that includes party i
            change_of_BS_i = sum(2**(steps - len(coalition_set)) for coalition_set in coalition_sets_with_i) #in words: for every set in coalition_sets_with_i calculate $2^{n-#U(...)}$ and add them all togehter
            if i % 2 == 1:  #  if i odd: add change_of_BS_i
                BS_i += change_of_BS_i
            else:  # if i even: substract change_of_BS_i
                BS_i -= change_of_BS_i
        BS[party_i]=BS_i      


In [4]:
banzaf_dict=BS_w(min_win_coals_year,relevant_parties_for_a_year)

KeyboardInterrupt: 